In [137]:
import sys
import os
import numpy as np
from collections import Counter
from collections import defaultdict
import json
from nltk.metrics import distance

def read_source(directory_path):
    source_arr = []
    for root, dirs, files in os.walk(directory_path):
        for f in files:
            with open(directory_path + "/" + f, "r") as source:
                s = source.read()
                source_arr.append((f, f.split('_')[0], s))
    return source_arr

def get_file_extensions():
    path = os.path.expanduser("~/Dropbox/classes/Fall 2017/project/data/{}/source/".format("AAN"))
    arr = read_source(path)
    print Counter([a[0].split('.')[-1] for a in arr])
    
get_file_extensions()

def read_json_file():
    path = os.path.expanduser("~/Dropbox/classes/Fall 2017/project/data/AAN/source_features.json")
    with open(path, "r") as f:
        file_list = json.loads(f.read())
    
    return file_list
    

def search_for_name(keyword, json_dict, type_filter = None):
    matches = []
    for file_name in json_dict.keys():
        for line in json_dict[file_name]:
            right_type = (type_filter is None or line["type"] == type_filter)
            if line["name"] == keyword and right_type:
                matches.append((file_name, line))
    return matches
    
    

def search_for_docstring(keyword, json_dict, type_filter = None):
    matches = []
    for file_name in json_dict.keys():
        for line in json_dict[file_name]:
            right_type = (type_filter is None or line["type"] == type_filter)
            if keyword in line["docstring"] and right_type:
                matches.append((file_name, line))
    return matches

def read_source(file_name):
    try:
        with open(file_name, "r") as f:
            source = f.read()
        return source
    except Exception:
        return ""
    

def print_source(file_name, line_start, line_end):
    source = read_source(file_name)
    return "\n".join(source.split("\n")[line_start - 1 :line_end])
        

def get_indentation(line_string):
    whitespace = [" ", "\t"]
    indentation = ""
    for char in line_string:
        if char in whitespace:
            indentation += char
        else:
            break
    return indentation

def infer_end_line(file_name, line_num):
    source = read_source(file_name)
    lines = source.split("\n")
    
    target_indentation = get_indentation(lines[line_num - 1])
    i = line_num
    while i < len(lines):
        line = lines[i]
        if (get_indentation(line) == target_indentation) and len(line) > 0:
            if i == line_num:
                return i
            return i - 1
        i += 1
    return i


def find_matches_by_name(json_dict, search_term, type_filter=None):
    matches = search_for_name(search_term, json_dict, type_filter)
    if len(matches):
        end =  infer_end_line(matches[0][0], matches[0][1])
        return print_source(matches[0][0], matches[0][1]["line"], end)
    else:
        return "No Matches Found."

    

def find_matches_by_docstring(json_dict, search_term, type_filter=None):
    matches = search_for_docstring(search_term, json_dict, type_filter)
    if len(matches):
        end =  infer_end_line(matches[0][0], matches[0][1])
        return print_source(matches[0][0], matches[0][1]["line"], end)
    else:
        return "No Matches Found."

json_dict = read_json_file()


Counter({'py': 10138, 'java': 4974, 'md': 1435, 'cpp': 902, 'c': 626, 'js': 376, 'pl': 73, 'r': 4})


In [150]:
# trees = defaultdict(list)
# roots = []
# path_stack = []
# for line in json_dict[json_dict.keys()[3]]:
#     if line["parent"] != "":
#         if line["parent"] != path_stack[-1]:
#             if line["parent"] in [c[1] for c in trees[path_stack[-1]]]:
#                 path_stack.append(line["parent"])
#             else:
#                 path_stack.pop()    

#         parent_path = "/".join(path_stack)
#         new_path = parent_path + "/" + line["name"]
#         child = (new_path, line["name"], line["type"], line["line"])
        
#         trees[parent_path].append(child)
#     else:
#         path_stack = [line["name"]]
#         roots.append((line["name"], line["name"], line["type"], line["line"]))


# def get_line_spans(node):
#     if len(trees[node[0]]) == 0:
# #         print node[0], (node[3], node[3])
#         return (node[3], node[3])
#     else:
#         children = trees[node[0]]
#         start = node[3]
#         end = -float("inf")
#         for c in children:
#             _, e = get_line_spans(c)
#             if e > end:
#                 end = e
# #         print (start, end)
#         return (start, end)
        

# for root in roots:
#     s,e = get_line_spans(root)
#     print print_source(json_dict.keys()[3], s,e)
#     print "---------------------------------------------------------------------"
    

In [166]:
functions = []
classes = []
for f in json_dict:
    for line in json_dict[f]:
        obj = (f, line["type"], line["name"], line["line"])
        if line["type"] == "class":
            classes.append(obj)
        elif line["type"] == "function":
            functions.append(obj)

functions_source = {}
classes_source = {}

for t in functions:
    try:
        end_line = infer_end_line(t[0], t[3])
        key = (t[0], t[1], t[2], t[3], end_line)
        functions_source[key] = print_source(key[0], key[3], end_line)
    except Exception:
        pass

for t in classes:
    try:
        end_line = infer_end_line(t[0], t[3])
        key = (t[0], t[1], t[2], t[3], end_line)
        classes_source[key] = print_source(key[0], key[3], end_line)
    except Exception:
        pass   

In [171]:
print functions_source[functions_source.keys()[7]]

    def params(self):
        """
        .. todo::

            WRITEME
        """
        return list(self._params)


[2,
 3,
 4,
 5,
 6,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 24,
 25,
 26,
 27,
 29,
 30,
 33,
 34,
 37,
 38,
 41,
 44,
 44,
 47,
 51,
 51,
 51,
 51,
 51,
 56,
 57,
 58,
 59,
 60,
 61,
 64,
 67,
 67,
 67,
 67,
 67,
 71,
 72,
 73,
 74,
 78,
 82,
 82,
 84,
 85,
 88,
 92,
 95,
 100,
 100,
 100,
 103,
 104,
 104,
 104,
 107,
 110,
 113,
 116,
 120,
 121,
 121,
 122,
 122,
 126,
 128,
 133,
 136,
 137,
 138,
 139,
 140,
 141,
 144,
 145,
 146,
 148,
 148,
 154,
 157,
 160,
 163,
 163,
 165,
 166,
 167,
 170,
 171,
 172,
 176,
 177,
 180,
 181,
 182,
 185,
 185,
 185,
 190,
 190,
 190,
 194,
 194,
 194,
 194,
 198,
 204,
 205,
 206,
 207,
 214,
 225,
 225,
 225,
 226,
 227,
 231,
 233,
 239,
 239,
 244,
 244,
 244,
 244,
 245,
 252,
 252,
 252,
 253,
 255,
 257,
 259,
 261]